In [2]:
import os
import sys


In [1]:
class Aclass:
    def __init__(self):
        print("Aclass.__init__()")
    
    def __str__(self):
        return "Aclass.__str__()"
    def __repr__(self):
        return "Aclass.__repr__()"

In [3]:
obj=Aclass()

Aclass.__init__()


In [5]:
print(obj)

Aclass.__str__()


In [4]:
obj

Aclass.__repr__()

In [ ]:
class TradingException(Exception):
    """Base class for all trading-related exceptions."""
    def __init__(self,error_message,error_deails:sys):
        self.error_message=error_message
        _,_,exc_tb=error_deails.exc_info()

        self.lineno=exc_tb.tb_lineno
        self.filename=exc_tb.tb_frame.f_code.co_filename

    def __str__(self):
        return "Error occured in python script name [{0}] line number [{1}] error message [{2}]".format(
            self.filename,self.lineno,str(self.error_message)
        )


In [6]:
import os
from langchain.tools import tool
from langchain_community.tools import TavilySearchResults
from langchain_community.tools.polygon.financials import PolygonFinancials
from langchain_community.utilities.polygon import PolygonAPIWrapper
from langchain_community.agent_toolkits.polygon.toolkit import PolygonToolkit
from langchain_community.tools.bing_search import BingSearchResults 
from data_models.models import RagToolSchema
from langchain_pinecone import PineconeVectorStore
from utils.model_loader import ModelLoader
from utils.config_loader import load_config
from dotenv import load_dotenv
from pinecone import Pinecone
import streamlit as st

In [9]:
load_dotenv()

api_wrapper = PolygonAPIWrapper()

In [37]:
api_key = os.getenv("POLYGON_API_KEY")
api_key

'pp4WVnnLM0ef9OlWPG1kd3x_GCzIGUf7'

In [12]:
from langchain_community.agent_toolkits.polygon.toolkit import PolygonToolkit
from langchain_community.utilities.polygon import PolygonAPIWrapper

polygon = PolygonAPIWrapper()
toolkit = PolygonToolkit.from_polygon_api_wrapper(polygon)

In [13]:
toolkit.get_tools()

[PolygonAggregates(api_wrapper=PolygonAPIWrapper(polygon_api_key='pp4WVnnLM0ef9OlWPG1kd3x_GCzIGUf7')),
 PolygonLastQuote(api_wrapper=PolygonAPIWrapper(polygon_api_key='pp4WVnnLM0ef9OlWPG1kd3x_GCzIGUf7')),
 PolygonTickerNews(api_wrapper=PolygonAPIWrapper(polygon_api_key='pp4WVnnLM0ef9OlWPG1kd3x_GCzIGUf7')),
 PolygonFinancials(api_wrapper=PolygonAPIWrapper(polygon_api_key='pp4WVnnLM0ef9OlWPG1kd3x_GCzIGUf7'))]

In [60]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm=ChatGoogleGenerativeAI(model="gemini-2.0-flash")

In [61]:
llm.invoke("hi")

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--549689cf-58f4-4508-87fb-7725ebf340ab-0', usage_metadata={'input_tokens': 1, 'output_tokens': 11, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

In [26]:
tool=toolkit.get_tools()

In [43]:
tool

[PolygonAggregates(api_wrapper=PolygonAPIWrapper(polygon_api_key='pp4WVnnLM0ef9OlWPG1kd3x_GCzIGUf7')),
 PolygonLastQuote(api_wrapper=PolygonAPIWrapper(polygon_api_key='pp4WVnnLM0ef9OlWPG1kd3x_GCzIGUf7')),
 PolygonTickerNews(api_wrapper=PolygonAPIWrapper(polygon_api_key='pp4WVnnLM0ef9OlWPG1kd3x_GCzIGUf7')),
 PolygonFinancials(api_wrapper=PolygonAPIWrapper(polygon_api_key='pp4WVnnLM0ef9OlWPG1kd3x_GCzIGUf7'))]

In [44]:

llm_with_tools = llm.bind_tools(tools=tool)



In [52]:
llm_with_tools.invoke( "fetch daily aggregates for GOOGL for last week")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'polygon_aggregates', 'arguments': '{"timespan_multiplier": 1.0, "ticker": "GOOGL", "to_date": "2024-05-17", "from_date": "2024-05-13", "timespan": "day"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--80e13215-aafc-4b16-8585-53ace2282b8b-0', tool_calls=[{'name': 'polygon_aggregates', 'args': {'timespan_multiplier': 1.0, 'ticker': 'GOOGL', 'to_date': '2024-05-17', 'from_date': '2024-05-13', 'timespan': 'day'}, 'id': 'cc9b7729-b5b9-4480-b0f4-5366f6984f7b', 'type': 'tool_call'}], usage_metadata={'input_tokens': 408, 'output_tokens': 42, 'total_tokens': 450, 'input_token_details': {'cache_read': 0}})

In [40]:
from langchain_community.tools.polygon.financials import PolygonFinancialsTool
financials_tool = PolygonFinancialsTool(polygon_api_key=api_key)

ImportError: cannot import name 'PolygonFinancialsTool' from 'langchain_community.tools.polygon.financials' (d:\Desktop\Research\krish\Projects\AgenticTradingAssistance\venv\Lib\site-packages\langchain_community\tools\polygon\financials.py)

In [42]:
from langchain_community.tools.polygon.financials import PolygonFinancials

financials_tool = PolygonFinancials(api_wrapper=api_wrapper)

# We can invoke directly with input
res = financials_tool.invoke({"query": "AAPL"})

model_generated_tool_call = {
    "args": {"query": "AAPL"},
    "id": "1",
    "name": financials_tool.name,
    "type": "tool_call",
}

# Or we can invoke with a tool call
res = financials_tool.invoke(model_generated_tool_call)

print(res)

content='[{"start_date": "2024-03-30", "end_date": "2025-03-29", "timeframe": "ttm", "fiscal_period": "TTM", "fiscal_year": "", "cik": "0000320193", "sic": "3571", "tickers": ["AAPL"], "company_name": "APPLE INC", "financials": {"balance_sheet": {"liabilities": {"value": 264437000000.0, "unit": "USD", "label": "Liabilities", "order": 600}, "fixed_assets": {"value": 46876000000.0, "unit": "USD", "label": "Fixed Assets", "order": 320}, "noncurrent_assets": {"value": 212559000000.0, "unit": "USD", "label": "Noncurrent Assets", "order": 300}, "accounts_payable": {"value": 54126000000.0, "unit": "USD", "label": "Accounts Payable", "order": 710}, "long_term_debt": {"value": 92200000000.0, "unit": "USD", "label": "Long-term Debt", "order": 810}, "noncurrent_liabilities": {"value": 119866000000.0, "unit": "USD", "label": "Noncurrent Liabilities", "order": 800}, "other_noncurrent_liabilities": {"value": 27666000000.0, "unit": "USD", "label": "Other Non-current Liabilities", "order": 820}, "liab

In [53]:
from langchain_community.agent_toolkits.polygon.toolkit import PolygonToolkit
from langchain_community.utilities.polygon import PolygonAPIWrapper
from dotenv import load_dotenv
import os

# Load environment variables and print API key status (don't print the actual key)
load_dotenv()
polygon_api_key = os.getenv("POLYGON_API_KEY")
print(f"API Key loaded: {'Yes' if polygon_api_key else 'No'}")

# Create API wrapper with explicit error handling
try:
    api_wrapper = PolygonAPIWrapper()
    polygon = PolygonAPIWrapper()
    toolkit = PolygonToolkit.from_polygon_api_wrapper(polygon)
    from langchain_google_genai import ChatGoogleGenerativeAI
    
    # Make sure your Google API key is also loaded
    google_api_key = os.getenv("GOOGLE_API_KEY")
    print(f"Google API Key loaded: {'Yes' if google_api_key else 'No'}")
    
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
    tools = toolkit.get_tools()
    llm_with_tools = llm.bind_tools(tools=tools)
    
    # Execute the query and explicitly print the result
    print("Fetching data...")
    result = llm_with_tools.invoke("fetch daily aggregates for GOOGL for last week")
    print("Result type:", type(result))
    print("Result content:")
    print(result)
    
    # If we got a result with tool calls, extract and display the actual data
    if hasattr(result, 'tool_calls') and result.tool_calls:
        print("\nTool calls detected:")
        for call in result.tool_calls:
            print(f"Tool: {call['name']}")
            print(f"Arguments: {call['args']}")
            
            # If using LangChain's tools directly, you could execute them here
            if call['name'] == 'polygon_aggregates':
                print("\nDirectly calling Polygon API...")
                direct_result = polygon.get_aggregates(
                    ticker=call['args']['ticker'],
                    from_date=call['args']['from_date'],
                    to_date=call['args']['to_date'],
                    timespan=call['args']['timespan'],
                    timespan_multiplier=call['args']['timespan_multiplier']
                )
                print("Direct API result:")
                print(direct_result)
                
except Exception as e:
    print(f"Error occurred: {e}")

API Key loaded: Yes
Google API Key loaded: Yes
Fetching data...
Result type: <class 'langchain_core.messages.ai.AIMessage'>
Result content:
content='' additional_kwargs={'function_call': {'name': 'polygon_aggregates', 'arguments': '{"timespan_multiplier": 1.0, "ticker": "GOOGL", "to_date": "2024-05-17", "from_date": "2024-05-13", "timespan": "day"}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--f62db56c-598f-4bb1-87d3-23b18f7f9f9b-0' tool_calls=[{'name': 'polygon_aggregates', 'args': {'timespan_multiplier': 1.0, 'ticker': 'GOOGL', 'to_date': '2024-05-17', 'from_date': '2024-05-13', 'timespan': 'day'}, 'id': '64117607-a0b0-4d1d-aeed-8552003ef01e', 'type': 'tool_call'}] usage_metadata={'input_tokens': 408, 'output_tokens': 42, 'total_tokens': 450, 'input_token_details': {'cache_read': 0}}

Tool calls detected:
Tool: polygon_aggregates
Arguments: {'timespan_multip

In [54]:
from langchain_community.agent_toolkits.polygon.toolkit import PolygonToolkit
from langchain_community.utilities.polygon import PolygonAPIWrapper
from dotenv import load_dotenv
import os

load_dotenv()
polygon_api_key = os.getenv("POLYGON_API_KEY")
print(f"API Key loaded: {'Yes' if polygon_api_key else 'No'}")

try:
    polygon = PolygonAPIWrapper()
    toolkit = PolygonToolkit.from_polygon_api_wrapper(polygon)
    
    # Get the specific aggregates tool
    tools = toolkit.get_tools()
    agg_tool = next(tool for tool in tools if tool.name == "polygon_aggregates")
    
    # Execute the tool directly with fixed parameters
    result = agg_tool.invoke({
        "ticker": "GOOGL",
        "from_date": "2024-05-13",
        "to_date": "2024-05-17",
        "timespan": "day",
        "timespan_multiplier": 1  # Use integer instead of float
    })
    
    print("Tool execution result:")
    print(result)
    
except Exception as e:
    print(f"Error occurred: {e}")

API Key loaded: Yes
Tool execution result:
[{"v": 31327590.0, "vw": 166.9002, "o": 164.26, "c": 169.14, "h": 169.28, "l": 164, "t": 1715572800000, "n": 423074}, {"v": 25127138.0, "vw": 170.1127, "o": 169.77, "c": 170.34, "h": 171.25, "l": 168.8, "t": 1715659200000, "n": 322375}, {"v": 26948370.0, "vw": 171.913, "o": 170.63, "c": 172.51, "h": 172.65, "l": 170.51, "t": 1715745600000, "n": 340477}, {"v": 27867947.0, "vw": 174.158, "o": 173.29, "c": 174.18, "h": 175.115, "l": 172.69, "t": 1715832000000, "n": 319821}, {"v": 24479262.0, "vw": 175.6884, "o": 174.18, "c": 176.06, "h": 176.265, "l": 173.69, "t": 1715918400000, "n": 277992}]


In [55]:
from langchain_community.agent_toolkits.polygon.toolkit import PolygonToolkit
from langchain_community.utilities.polygon import PolygonAPIWrapper
from dotenv import load_dotenv
import os

load_dotenv()
polygon_api_key = os.getenv("POLYGON_API_KEY")
print(f"API Key loaded: {'Yes' if polygon_api_key else 'No'}")

try:
    api_wrapper = PolygonAPIWrapper()
    polygon = PolygonAPIWrapper()
    toolkit = PolygonToolkit.from_polygon_api_wrapper(polygon)
    from langchain_google_genai import ChatGoogleGenerativeAI
    
    google_api_key = os.getenv("GOOGLE_API_KEY")
    print(f"Google API Key loaded: {'Yes' if google_api_key else 'No'}")
    
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
    tools = toolkit.get_tools()
    llm_with_tools = llm.bind_tools(tools=tools)
    
    print("Fetching data...")
    result = llm_with_tools.invoke("fetch daily aggregates for GOOGL for last week")
    print("Result type:", type(result))
    print("Result content:")
    print(result)
    
    if hasattr(result, 'tool_calls') and result.tool_calls:
        print("\nTool calls detected:")
        for call in result.tool_calls:
            print(f"Tool: {call['name']}")
            print(f"Arguments: {call['args']}")
            
            if call['name'] == 'polygon_aggregates':
                args = call['args']
                print("\nDirectly calling Polygon API...")
                
                # Convert timespan_multiplier to integer
                timespan_multiplier = int(args['timespan_multiplier'])
                
                direct_result = polygon.get_aggregates(
                    ticker=args['ticker'],
                    from_date=args['from_date'],
                    to_date=args['to_date'],
                    timespan=args['timespan'],
                    timespan_multiplier=timespan_multiplier  # Use integer instead of float
                )
                print("Direct API result:")
                print(direct_result)
                
except Exception as e:
    print(f"Error occurred: {e}")

API Key loaded: Yes
Google API Key loaded: Yes
Fetching data...
Result type: <class 'langchain_core.messages.ai.AIMessage'>
Result content:
content='' additional_kwargs={'function_call': {'name': 'polygon_aggregates', 'arguments': '{"timespan_multiplier": 1.0, "ticker": "GOOGL", "to_date": "2024-05-17", "from_date": "2024-05-13", "timespan": "day"}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--d2b4e6d0-094e-4a78-b084-4977e3ba9468-0' tool_calls=[{'name': 'polygon_aggregates', 'args': {'timespan_multiplier': 1.0, 'ticker': 'GOOGL', 'to_date': '2024-05-17', 'from_date': '2024-05-13', 'timespan': 'day'}, 'id': 'ed92194b-632f-41ff-ae8b-f0d463baed5c', 'type': 'tool_call'}] usage_metadata={'input_tokens': 408, 'output_tokens': 42, 'total_tokens': 450, 'input_token_details': {'cache_read': 0}}

Tool calls detected:
Tool: polygon_aggregates
Arguments: {'timespan_multip

In [56]:
from langchain_community.tools import TavilySearchResults

In [64]:
tavily_api_key = os.getenv("TAVILY_API_KEY")
tavily_api_key

'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'

In [62]:
def tavily_tool(question: str):
    """Use this tool to search the web for recent or specific information."""
    try:
        tavily_api_key = os.getenv("TAVILY_API_KEY")
        if not tavily_api_key:
            return "Error: Tavily API key not found."
        
        search_tool = TavilySearchResults(
            api_key=tavily_api_key, 
            max_results=5,
            search_depth="advanced",
            include_answer=True,
            include_raw_content=True
        )
        
        results = search_tool.invoke(question)
        return results
    except Exception as e:
        return f"Error using Tavily search: {str(e)}. Please ensure the Tavily API key is valid and the service is available."

In [63]:
tavily_tool("ai news")

[{'title': 'AI News | Latest AI News, Analysis & Events',
  'url': 'https://www.artificialintelligence-news.com/',
  'content': 'The technical storage or access is required to create user profiles to send advertising, or to track the user on a website or across several websites for similar marketing purposes.\nManage options Manage services Manage {vendor_count} vendors Read more about these purposes\nAccept Deny View preferences Save preferences View preferences\nCookie Policy Privacy Policy {title}\nSkip to content\nAI News is part of the TechForge Publications series [...] ServiceNow bets on unified AI to untangle enterprise complexity\nApplications\nMay 7, 2025\nMachine Learning\nOpenAI’s latest LLM opens doors for China’s AI startups\nAlibaba\nApril 29, 2025\nRed Hat on open, small language models for responsible, practical AI\nArtificial Intelligence\nApril 22, 2025\nTony Blair Institute AI copyright report sparks backlash\nAGI\nApril 2, 2025 [...] Applications\nMay 2, 2025\nDeep

In [ ]:
llm.